# Run CellProfiler `illum.cppipe` (IC) pipeline

In this notebook, we run the CellProfiler IC pipeline to calculate the illumination (illum) correction functions for all images per channel (5), apply the functions, and save images into a new directory.

## Import libraries

In [1]:
import argparse
import pathlib
import sys
import time

sys.path.append("../../../utils")
import cp_parallel
import cp_utils as cp_utils
import tqdm

# check if in a jupyter notebook
try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

## Set paths

In [2]:
if not in_notebook:
    print("Running as script")
    # set up arg parser
    parser = argparse.ArgumentParser(description="Segment the nuclei of a tiff image")

    parser.add_argument(
        "--input_dir",
        type=str,
        help="Path to the input directory containing the tiff images",
    )

    args = parser.parse_args()
    input_dir = pathlib.Path(args.input_dir).resolve(strict=True)
else:
    print("Running in a notebook")
    input_dir = pathlib.Path("../../../data/test_dir/").resolve(strict=True)

run_name = "illumination_correction"
# path to folder for IC images
illum_directory = pathlib.Path("../illum_directory").resolve()
# make sure the directory exists
illum_directory.mkdir(exist_ok=True, parents=True)

Running in a notebook


## Define the input paths

5 FOVs per well, 96 wells per plate, 1 plate at 18 time points = 8640 image sets

In [3]:
path_to_pipeline = pathlib.Path("../pipelines/illum_5ch.cppipe").resolve(strict=True)
# get all directories with raw images
dict_of_runs = {}
raw_directories = list(input_dir.glob("*"))
raw_directories = [x for x in raw_directories if x.is_dir()]
raw_directories = sorted(raw_directories)


for dir in raw_directories:
    well_FOV = dir.name
    print(dir.name)
    print(dir)
    dict_of_runs[well_FOV] = {
        "path_to_images": str(dir),
        "path_to_output": illum_directory / well_FOV,
        "path_to_pipeline": path_to_pipeline,
    }
print(f"Added {len(dict_of_runs.keys())} to the list of runs")

W0052_F0001
/home/lippincm/Documents/pyroptosis_live-cell_timelapse/data/test_dir/W0052_F0001
Added 1 to the list of runs


## Run `illum.cppipe` pipeline and calculate + save IC images
This last cell does not get run as we run this pipeline in the command line.

In [4]:
start = time.time()

In [5]:
cp_parallel.run_cellprofiler_parallel(
    plate_info_dictionary=dict_of_runs, run_name=run_name
)

Number of processes: 1
All processes have been completed!
1
A return code of 1 was returned for CompletedProcess(args=['cellprofiler', '-c', '-r', '-p', PosixPath('/home/lippincm/Documents/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/pipelines/illum_5ch.cppipe'), '-o', PosixPath('/home/lippincm/Documents/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001'), '-i', '/home/lippincm/Documents/pyroptosis_live-cell_timelapse/data/test_dir/W0052_F0001'], returncode=1, stdout=b'CP-JAVA 21:28:03.553 [Thread-0] WARN  o.c.imageset.ChannelFilter - Empty image set list: no images passed the filtering criteria.\n', stderr=b''), which means there was an error in the CellProfiler run.
All results have been converted to log files!


In [6]:
end = time.time()
# format the time taken into hours, minutes, seconds
hours, rem = divmod(end - start, 3600)
minutes, seconds = divmod(rem, 60)
print(
    "Total time taken: {:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds)
)

Total time taken: 00:00:01.47
